Drill Hole Compound Likelihood Demo
===================================

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as pl
import numpy as np
import logging

import revrand.optimize as opt
from revrand import glm
from revrand.basis_functions import BiasBasis, RandomRBF
from revrand.btypes import Parameter, Positive
from revrand.utils.datasets import gen_gausprocess_se
from revrand.mathfun.special import softplus

from uncoverml.likelihoods import Switching, PosGaussian

# Log output to the terminal attached to this notebook
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

# Warning handling
#np.seterr(all='warn')
import warnings
warnings.filterwarnings('error')


Demo settings
-------------

In [3]:
# Dataset properties
N = 300
Ns = 250
offset = 20
lenscale_true = 1.2  # For the gpdraw dataset
noise_true = 0.1
hit_proportion = 0.0

fig_width = 15
fig_height = 10


Algorithm Settings
------------------

In [4]:
nbases = 20
lenscale = 1  # For all basis functions that take lengthscales
maxiter = 3000
batch_size = 10
use_sgd = True
regulariser = 1.
noise = 1.
#updater = opt.AdaDelta(rho=0.1, epsilon=1e-6)
updater = opt.Adam(alpha=0.01, epsilon=1e-5, beta1=0.1, beta2=0.3)


Make data
---------

In [5]:
hit = np.random.binomial(n=1, p=hit_proportion, size=N).astype(bool)
not_hit = ~ hit
Xtrain, ftrain, Xtest, ftest = \
    gen_gausprocess_se(N, Ns, lenscale=lenscale_true, noise=0.0)

gtrain = ftrain + offset
gtest = ftest + offset

ytrain = np.empty(N)
ytrain[hit] = gtrain[hit] + np.random.randn(hit.sum()) * noise_true
ytrain[not_hit] = np.random.rand(not_hit.sum()) * gtrain[not_hit]


Set up likelihood and Basis
---------------------------

In [6]:
# Make parameter types
var = Parameter(noise**2, Positive())
regulariser = Parameter(regulariser, Positive())
lenscale_init = Parameter(lenscale, Positive(10))

# Likelihood
learn_args = (hit,)
pred_args = (np.ones(Ns, dtype=bool),)
like = Switching(var_init=var)

#learn_args = ()
#pred_args = ()
#like = PosGaussian(var_init=var)

# Regression Basis
basis = BiasBasis(offset=offset) \
    + RandomRBF(nbases, Xtrain.shape[1], lenscale_init)


Learning and prediction
-----------------------

In [7]:
# Learning
params = glm.learn(Xtrain, ytrain, like, basis, regulariser=regulariser,
                   likelihood_args=learn_args, use_sgd=use_sgd,
                   batch_size=batch_size, maxiter=maxiter, updater=updater)

# Expected drill hole measurements (proxy for basement depth)
Ey, Vy, Eyn, Eyx = glm.predict_moments(Xtest, like, basis, *params,
                                       likelihood_args=pred_args)

# 95% confidence interval
y95n, y95x = glm.predict_interval(0.8, Xtest, like, basis, *params,
                                  multiproc=False,
                                  likelihood_args=pred_args)


fmin: -40.97767274868346, fmax: -36.03001300440637, fshape: (10,)
fmin: -17.0854949717672, fmax: -16.119863722342835, fshape: (10,)
fmin: -0.5618639565510158, fmax: 1.049889778874959, fshape: (10,)
fmin: -2.452224271256076, fmax: 0.26269140358423126, fshape: (10,)
fmin: -3.362269246635897, fmax: 2.1324159722489866, fshape: (10,)
fmin: -2.8635250780849635, fmax: 4.426558964479066, fshape: (10,)
fmin: -7.040373321386459, fmax: 4.516837730603239, fshape: (10,)
fmin: -8.568631784899479, fmax: 5.704256016983359, fshape: (10,)
fmin: -7.854714949379624, fmax: 7.136993095719642, fshape: (10,)
fmin: -9.598769730859994, fmax: 7.973986482621383, fshape: (10,)
fmin: -8.51854868686283, fmax: 6.667264615204894, fshape: (10,)
fmin: -10.410194078575731, fmax: 4.05774388727976, fshape: (10,)
fmin: -7.387527611076125, fmax: 7.527732383807039, fshape: (10,)
fmin: -10.447084236230811, fmax: 4.730685856225785, fshape: (10,)
fmin: -6.937001798890388, fmax: 5.442258586433467, fshape: (10,)
fmin: -10.21953758

KeyboardInterrupt: 

Visualise
---------

In [ ]:
Xpl_t = Xtrain.flatten()
Xpl_s = Xtest.flatten()

# Regressor
pl.figure(figsize=(fig_width, fig_height))
pl.plot(Xpl_s, Ey, 'b-', label='GLM mean.')
pl.fill_between(Xpl_s, y95n, y95x, facecolor='none', edgecolor='b', label=None,
                linestyle='--')

# # Training/Truth
pl.plot(Xpl_t[hit], ytrain[hit], 'k.', label='Training (hit)')
pl.plot(Xpl_t[not_hit], ytrain[not_hit], 'kx', label='Training (not hit)')
pl.plot(Xpl_s, gtest, 'k-', label='Truth')

pl.gca().invert_yaxis()
pl.legend()
pl.grid(True)
pl.title('Regression demo')
pl.ylabel('depth')
pl.xlabel('x')

pl.show()
